<h1>Segmenting and Clustering the Neighborhood in Toronto | Part 3</h1>

<h3>Import Libraries</h3>

In [1]:
import numpy as np
import pandas as pd

from geopy.geocoders import Nominatim       # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [2]:
toronto_df = pd.read_csv('toronto_updated.csv')
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<h3>Use geopy library to obtain the latitude and longitude values of Toronto</h3>

In [3]:
address = 'Toronto'

geolocator = Nominatim(user_agent= "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {a} and {b}'.format(a= latitude, b= longitude))

The geographical coordinates of Toronto are 43.6534817 and -79.3839347


<h3>Create a map of Toronto with neighborhoods superimposed on top</h3>

In [4]:
# Create the map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Filter boroughs that contain the word Toronto

In [5]:
# Filter the borough names that contains the word 'Toronto'
borough_names = list(toronto_df['Borough'].unique())

borough_with_toronto = []

for borough_name in borough_names:
    if 'toronto' in borough_name.lower():
        borough_with_toronto.append(borough_name)
    
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [6]:
# Let's create a new Dataframe with Boroughs contains the name 'Toronto'
toronto_df = toronto_df[toronto_df['Borough'].isin(borough_with_toronto)].reset_index(drop= True)
print(toronto_df.shape)
toronto_df.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [7]:
# Create map of Toronto using Latitude and Longitude values
map_toronto = folium.Map(location= [latitude, longitude], zoom_start=12)

# Add marker to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html= True)
    folium.CircleMarker([lat, lng],
                       radius= 5,
                       popup= label,
                       color= 'blue',
                       fill= True,
                       fill_color= '#3186cc',
                       fill_opacity= 0.7).add_to(map_toronto)
    
map_toronto

### Use Foursquare API to explore the neighborhoods

In [8]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'L3XLTZ2HARAD1BNWETR0JZYF3C3K0HYGEV4I3MV1A30EOSEF' # your Foursquare ID
CLIENT_SECRET = 'QDHJGUKOJQ02VRG3HNQWS3PE3NJFFOIIPAEWTN4XEEGPIAED' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:\n')
print('CLIENT_ID: ' + CLIENT_ID)
print('\nCLIENT_SECRET: ' + CLIENT_SECRET)

Your credentails:

CLIENT_ID: L3XLTZ2HARAD1BNWETR0JZYF3C3K0HYGEV4I3MV1A30EOSEF

CLIENT_SECRET: QDHJGUKOJQ02VRG3HNQWS3PE3NJFFOIIPAEWTN4XEEGPIAED


In [9]:
radius = 500
LIMIT = 100

venues = []

for lat, lng, postalcode, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['PostalCode'], toronto_df['Borough'], toronto_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
    
    results = requests.get(url).json()['response']['groups'][0]['items']
    
    for venue in results:
        venues.append((postalcode, borough, neighborhood, lat, lng, 
                       venue['venue']['name'], 
                       venue['venue']['location']['lat'], 
                       venue['venue']['location']['lng'],
                       venue['venue']['categories'][0]['name']))

In [10]:
# Let's convert the venues list into a new dataframe
venues_df = pd.DataFrame(data = venues, columns=['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory'])

print(venues_df.shape)
venues_df.head()

(1610, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [11]:
venues_df.groupby(by= ['PostalCode', 'Borough', 'Neighborhood']).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West, Riverdale",41,41,41,41,41,41
M4L,East Toronto,"The Beaches West, India Bazaar",20,20,20,20,20,20
M4M,East Toronto,Studio District,36,36,36,36,36,36
M4N,Central Toronto,Lawrence Park,3,3,3,3,3,3
M4P,Central Toronto,Davisville North,9,9,9,9,9,9
M4R,Central Toronto,North Toronto West,22,22,22,22,22,22
M4S,Central Toronto,Davisville,33,33,33,33,33,33
M4T,Central Toronto,"Moore Park, Summerhill East",4,4,4,4,4,4


<h3>Find out how many unique categories can be created from all the returned venues</h3>

In [12]:
print('There are {} unique categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 235 unique categories.


In [13]:
venues_df['VenueCategory'].unique()[:50]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Cosmetics Shop', 'Greek Restaurant', 'Ice Cream Shop',
       'Italian Restaurant', 'Yoga Studio', 'Fruit & Vegetable Store',
       'Brewery', 'Dessert Shop', 'Restaurant', 'Juice Bar',
       'Pizza Place', 'Bookstore', 'Furniture / Home Store',
       'Bubble Tea Shop', 'Spa', 'Coffee Shop', 'Grocery Store', 'Bakery',
       'Tibetan Restaurant', 'Indian Restaurant', 'Caribbean Restaurant',
       'Café', 'Lounge', 'Frozen Yogurt Shop', 'Liquor Store',
       'American Restaurant', 'Gym', 'Fast Food Restaurant',
       'Fish & Chips Shop', 'Sushi Restaurant', 'Park', 'Pet Store',
       'Steakhouse', 'Movie Theater', 'Sandwich Place',
       'Food & Drink Shop', 'Fish Market', 'Gay Bar', 'Cheese Shop',
       'Seafood Restaurant', 'Comfort Food Restaurant',
       'Middle Eastern Restaurant', 'Stationery Store', 'Thai Restaurant',
       'Coworking Space', 'Latin American Restaurant'], dtype=object)

# Analyze Each Area

In [14]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1610, 238)


,PostalCode,Borough,Neighborhoods,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West, Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [15]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 238)


,PostalCode,Borough,Neighborhoods,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,...,0.00000,0.000000,0.25000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,...,0.02439,0.000000,0.02439,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390
2,M4L,East Toronto,"The Beaches West, India Bazaar",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M4M,East Toronto,Studio District,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778
4,M4N,Central Toronto,Lawrence Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M4P,Central Toronto,Davisville North,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4R,Central Toronto,North Toronto West,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455
7,M4S,Central Toronto,Davisville,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,...,0.00000,0.030303,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,...,0.00000,0.000000,0.25000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000


### Create the new dataframe and display the top 10 venues for each PostalCode

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Health Food Store,Pub,Neighborhood,Trail,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Liquor Store,Indian Restaurant,Spa,Bookstore,Brewery
2,M4L,East Toronto,"The Beaches West, India Bazaar",Park,Sandwich Place,Fast Food Restaurant,Movie Theater,Restaurant,Italian Restaurant,Fish & Chips Shop,Steakhouse,Food & Drink Shop,Ice Cream Shop
3,M4M,East Toronto,Studio District,Coffee Shop,Bakery,Gastropub,American Restaurant,Brewery,Café,Yoga Studio,Cheese Shop,Fish Market,Italian Restaurant
4,M4N,Central Toronto,Lawrence Park,Park,Bus Line,Swim School,Yoga Studio,Diner,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
5,M4P,Central Toronto,Davisville North,Department Store,Gym / Fitness Center,Park,Breakfast Spot,Pizza Place,Playground,Food & Drink Shop,Sandwich Place,Hotel,Dog Run
6,M4R,Central Toronto,North Toronto West,Coffee Shop,Clothing Store,Shoe Store,Seafood Restaurant,Salon / Barbershop,Restaurant,Rental Car Location,Café,Chinese Restaurant,Park
7,M4S,Central Toronto,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Italian Restaurant,Gym,Café,Coffee Shop,Sushi Restaurant,Park,Pharmacy
8,M4T,Central Toronto,"Moore Park, Summerhill East",Park,Restaurant,Gym,Trail,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Coffee Shop,American Restaurant,Supermarket,Sushi Restaurant,Light Rail Station,Fried Chicken Joint,Bank,Restaurant,Bagel Shop,Pub


## Cluster Areas using K means algorithm
Run k-means to cluster the Toronto areas into 5 clusters

In [17]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 0, 0, 0, 4, 0, 0, 0, 2, 0])

In [18]:
# Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
toronto_merged = toronto_df.copy()

# Add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# Merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(['Borough', 'Neighborhoods'], 1).set_index('PostalCode'), on= 'PostalCode')

print(toronto_merged.shape)
toronto_merged.head()        # Check last columns

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Health Food Store,Pub,Neighborhood,Trail,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Liquor Store,Indian Restaurant,Spa,Bookstore,Brewery
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Fast Food Restaurant,Movie Theater,Restaurant,Italian Restaurant,Fish & Chips Shop,Steakhouse,Food & Drink Shop,Ice Cream Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Bakery,Gastropub,American Restaurant,Brewery,Café,Yoga Studio,Cheese Shop,Fish Market,Italian Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Bus Line,Swim School,Yoga Studio,Diner,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [19]:
# Let's sort the results by 'Cluster labels'
toronto_merged.sort_values(by= ['Cluster Labels'], axis= 0, inplace= True)

print(toronto_merged.shape)
toronto_merged

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,0,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Sporting Goods Shop,Fried Chicken Joint,Brewery,Restaurant,Scenic Lookout
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0,Coffee Shop,Restaurant,Hotel,Café,Italian Restaurant,Gym,Japanese Restaurant,Seafood Restaurant,Cocktail Bar,Deli / Bodega
23,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,0,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Yoga Studio,Diner,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Café,Sandwich Place,Coffee Shop,History Museum,Indian Restaurant,Pub,Liquor Store,Middle Eastern Restaurant,BBQ Joint,Pizza Place
25,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049,0,Café,Bakery,Bar,Japanese Restaurant,Bookstore,Bank,Beer Bar,Beer Store,Sandwich Place,Restaurant
26,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,0,Café,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Grocery Store,Dessert Shop,Burger Joint,Park,Gaming Cafe
27,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,0,Airport Lounge,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry,Boutique
28,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,0,Coffee Shop,Seafood Restaurant,Cocktail Bar,Japanese Restaurant,Café,Beer Bar,Hotel,Bakery,Restaurant,Italian Restaurant
29,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0,Coffee Shop,Café,Hotel,Restaurant,Gym,Japanese Restaurant,Deli / Bodega,Salad Place,Asian Restaurant,Steakhouse
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Coffee Shop,Restaurant,Italian Restaurant,Baby Store,Nightclub,Candy Store,Doner Restaurant


### Visualize the resulting clusters

In [20]:
# Create map
map_clusters = folium.Map(location= [latitude, longitude], zoom_start= 12)

# Set color scheme for clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lng, neighborhood, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(neighborhood) + ' Cluster ' + str(cluster), parse_html= True)
    folium.CircleMarker([lat, lng],
                        radius= 5,
                        popup= label,
                        color= rainbow[cluster-1],
                        fill= True,
                        fill_color= rainbow[cluster-1],
                        fill_opacity=0.7).add_to(map_clusters)
     
# map_clusters.save('Segmenting-and-clustering-neighborhood-in-toronto.html')
map_clusters  

## Examine Clusters

#### Cluster 1

In [21]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Sporting Goods Shop,Fried Chicken Joint,Brewery,Restaurant,Scenic Lookout
21,Downtown Toronto,0,Coffee Shop,Restaurant,Hotel,Café,Italian Restaurant,Gym,Japanese Restaurant,Seafood Restaurant,Cocktail Bar,Deli / Bodega
23,Central Toronto,0,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Yoga Studio,Diner,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
24,Central Toronto,0,Café,Sandwich Place,Coffee Shop,History Museum,Indian Restaurant,Pub,Liquor Store,Middle Eastern Restaurant,BBQ Joint,Pizza Place
25,Downtown Toronto,0,Café,Bakery,Bar,Japanese Restaurant,Bookstore,Bank,Beer Bar,Beer Store,Sandwich Place,Restaurant
26,Downtown Toronto,0,Café,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Grocery Store,Dessert Shop,Burger Joint,Park,Gaming Cafe
27,Downtown Toronto,0,Airport Lounge,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry,Boutique
28,Downtown Toronto,0,Coffee Shop,Seafood Restaurant,Cocktail Bar,Japanese Restaurant,Café,Beer Bar,Hotel,Bakery,Restaurant,Italian Restaurant
29,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Gym,Japanese Restaurant,Deli / Bodega,Salad Place,Asian Restaurant,Steakhouse
30,Downtown Toronto,0,Grocery Store,Café,Park,Coffee Shop,Restaurant,Italian Restaurant,Baby Store,Nightclub,Candy Store,Doner Restaurant


#### Cluster 2

In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Fast Food Restaurant,Home Service,Pool,Garden,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 3

In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,2,Park,Restaurant,Gym,Trail,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
10,Downtown Toronto,2,Park,Playground,Trail,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Cluster 4

In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,3,Health Food Store,Pub,Neighborhood,Trail,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant


#### Cluster 5

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,4,Park,Bus Line,Swim School,Yoga Studio,Diner,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


# Observation

Most of the neighborhoods fall into Cluster 1 which are mostly business areas with cafe, restaurants, supermarkets etc. Cluster 2 are (Restaurants, Pool & Garden) , Cluster 3 are (Park, Trail & Donut Shop) Cluster 4 (Restaurants, Pub, Studio & Store), and lastly Cluster 5 are (Bus line, Swim School & Escape Room).